# LangChain Text Splitters & Embeddings — Practical Workshop

**Phase:** Semantic Chunking + Embeddings + Gemini demo 💡📌🧠🔍

This notebook teaches practical splitters (including a semantic chunker you can use without `langchain_experimental`), shows how to generate embeddings, and demonstrates calling Gemini for contract-aware QA. It's Colab-ready and beginner friendly.

## Learning Guide

**What you'll learn**

- Why text splitting matters for legal/contract docs
- Three practical splitters and a semantic chunker implementation (no `langchain_experimental` required)
- How to get embeddings and compare similarity
- How to call Gemini (Google Generative) with chunked context

**Hands-on steps**

1. Install dependencies if needed
2. Load sample contract text
3. Run character/recursive/token splitters
4. Run the semantic chunker implemented here (embedding + greedy grouping)
5. Generate embeddings with Gemini or SentenceTransformers
6. Send best chunks to Gemini for an answer


In [1]:
from secrete_key import my_gemini_api_key
API_KEY = my_gemini_api_key()
print('API key loaded (hidden)')

API key loaded (hidden)


## Install / prerequisites

Run the next cell if you need to install packages in Colab / your environment. It installs (optionally) langchain, sentence-transformers, sklearn, and the Google LangChain integration.

It also avoids relying on `langchain_experimental` by providing a pure-Python semantic chunker implementation below.

In [2]:
# Sample contract-like text for demonstration
sample_contract = """SERVICE AGREEMENT

This Service Agreement ("Agreement") is made as of January 1, 2024, by and between Alpha Corp ("Provider") and Beta LLC ("Client").
1. Services. Provider shall provide software development services (the "Services") described in Schedule A.
2. Term. The term of this Agreement begins on the Effective Date and continues for twelve (12) months, unless earlier terminated.
3. Payment. Client will pay Provider the fees set forth in Schedule B within thirty (30) days of invoice receipt.
4. Confidentiality. Each party shall maintain confidential information in strict confidence and not disclose it to third parties.
5. Liability. Neither party shall be liable for indirect or consequential damages.
6. Termination. Either party may terminate upon thirty (30) days notice for material breach.
7. Governing Law. This Agreement shall be governed by the laws of the State of Delaware.
8. Miscellaneous. This Agreement constitutes the entire agreement between the parties and supersedes prior discussions.
"""
print('Loaded sample contract ({} chars)'.format(len(sample_contract)))

Loaded sample contract (1018 chars)


## Classic LangChain Splitters

The notebook will attempt to import LangChain splitters. If you don't have LangChain installed, either install it or skip to the semantic chunker (which does not require langchain_experimental).

In [8]:
pip install --quiet langchain

Note: you may need to restart the kernel to use updated packages.


In [10]:
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, MarkdownHeaderTextSplitter, TokenTextSplitter
    print('LangChain splitters imported.')
except Exception as e:
    print('Could not import LangChain splitters —', e)
    print('If you want them, run: pip install --quiet langchain')

Could not import LangChain splitters — No module named 'langchain.text_splitter'
If you want them, run: pip install --quiet langchain


In [ ]:
try:
    splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50, separators=["\n\n","\n"," ",""])
    rec_chunks = splitter.split_text(sample_contract)
    print('RecursiveCharacterTextSplitter ->', len(rec_chunks), 'chunks')
    for i,c in enumerate(rec_chunks):
        print(f'--- chunk {i+1} (len={len(c)}) ---\n{c[:200]}\n')
except Exception as e:
    print('Skipping recursive splitter:', e)

In [ ]:
try:
    from langchain.text_splitter import CharacterTextSplitter
    char_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=40)
    char_chunks = char_splitter.split_text(sample_contract)
    print('CharacterTextSplitter ->', len(char_chunks), 'chunks')
except Exception as e:
    print('Skipping character splitter:', e)

In [ ]:
try:
    from langchain.text_splitter import TokenTextSplitter
    token_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=30)
    token_chunks = token_splitter.split_text(sample_contract)
    print('TokenTextSplitter ->', len(token_chunks), 'chunks')
except Exception as e:
    print('Skipping token splitter:', e)

## SemanticChunker — custom implementation (no langchain_experimental)

This cell implements a semantic chunker using `sentence-transformers` embeddings + a greedy grouping algorithm. The idea:

1. Split the document into sentences.
2. Compute sentence embeddings.
3. Greedily group consecutive sentences until the group's embedding similarity to the next sentence is low or group reaches max characters.

This approach preserves clause boundaries and groups semantically coherent text without requiring a special LangChain experimental package.

In [ ]:
# SemanticChunker implementation (greedy grouping)
from sentence_transformers import SentenceTransformer
import numpy as np
import re

# Simple sentence splitter
def split_sentences(text):
    # naive splitter that keeps clause structure; for production use a proper sentence tokenizer
    sentences = re.split(r'(?<=[.!?\n])\s+', text.strip())
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

# cosine similarity
def cosine_sim(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10))

class SemanticChunker:
    def __init__(self, model_name='all-MiniLM-L6-v2', max_chars=800, min_chars=200, sim_threshold=0.7):
        self.model = SentenceTransformer(model_name)
        self.max_chars = max_chars
        self.min_chars = min_chars
        self.sim_threshold = sim_threshold
    
    def chunk(self, text):
        sentences = split_sentences(text)
        embeddings = self.model.encode(sentences, normalize_embeddings=True)
        chunks = []
        cur_sentences = []
        cur_embs = []
        for i, sent in enumerate(sentences):
            cur_sentences.append(sent)
            cur_embs.append(embeddings[i])
            cur_text = ' '.join(cur_sentences)
            # stop growing if too long
            if len(cur_text) >= self.max_chars:
                chunks.append(cur_text)
                cur_sentences = []
                cur_embs = []
                continue
            # lookahead similarity: if next sentence is dissimilar, close the chunk
            if i+1 < len(sentences):
                next_emb = embeddings[i+1]
                # compute centroid of current group
                centroid = np.mean(np.vstack(cur_embs), axis=0)
                sim = cosine_sim(centroid, next_emb)
                if sim < self.sim_threshold and len(cur_text) >= self.min_chars:
                    chunks.append(cur_text)
                    cur_sentences = []
                    cur_embs = []
        # flush remaining
        if cur_sentences:
            chunks.append(' '.join(cur_sentences))
        return chunks

# Demo
print('Loading sentence-transformers model (may download if not cached)...')
chunker = SemanticChunker(max_chars=600, min_chars=120, sim_threshold=0.65)
sem_chunks = chunker.chunk(sample_contract)
print('SemanticChunker produced', len(sem_chunks), 'chunks')
for i,c in enumerate(sem_chunks):
    print(f'--- chunk {i+1} (len={len(c)}) ---\n{c}\n')

## Sending semantic chunks to Gemini (example)

The following cell demonstrates how you'd send the top chunk(s) to the Gemini model using the `langchain_google_genai.ChatGoogleGenerativeAI` wrapper (matching your earlier code). It includes graceful fallbacks if the package isn't installed.

In [ ]:
try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=API_KEY)
    print('Model initialized!')
    # choose top N chunks (e.g., 2) to provide as context
    context = "\n\n".join(sem_chunks[:2])
    prompt = f"""You are a legal assistant. Using the context below from a service agreement, answer succinctly.\nContext:\n{context}\n\nQuestion: Summarize the key obligations of the Provider and Client in 3 bullet points."""
    response = model.invoke(prompt)
    print('\n=== Model response ===\n')
    print(response)
except Exception as e:
    print('Could not run Gemini invocation —', e)
    print('Make sure langchain-google-genai is installed and that API_KEY is valid.')

## Part 3 — Embeddings (Gemini + local alternatives)

This section shows how to get embeddings using either Gemini (via LangChain's GoogleGenerativeAIEmbeddings) or locally using `sentence-transformers`. Use Gemini embeddings in prod if you prefer provider-managed vectors; use sentence-transformers for offline demos.

In [ ]:
# Gemini embeddings (requires langchain_google_genai)
try:
    from langchain_google_genai import GoogleGenerativeAIEmbeddings
    gem_emb = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=API_KEY)
    texts = ['Terminate the contract immediately.', 'End the agreement right away.', 'The weather is sunny today.']
    vectors = gem_emb.embed_documents(texts)
    print('Got', len(vectors), 'embeddings from Gemini (length of first):', len(vectors[0]))
except Exception as e:
    print('Gemini embeddings not available —', e)
    print('Falling back to sentence-transformers for local embedding demo.')
    from sentence_transformers import SentenceTransformer
    import numpy as np
    st = SentenceTransformer('all-MiniLM-L6-v2')
    texts = ['Terminate the contract immediately.', 'End the agreement right away.', 'The weather is sunny today.']
    vectors = st.encode(texts, normalize_embeddings=True)
    def cosine_sim(a,b): 
        return float(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-10))
    print('Local embeddings ready. Similarities:')
    print('text1 vs text2:', cosine_sim(vectors[0], vectors[1]))
    print('text1 vs text3:', cosine_sim(vectors[0], vectors[2]))

## Best practices & troubleshooting

- If you saw `ModuleNotFoundError: No module named 'langchain_experimental'`, this notebook's SemanticChunker bypasses that by using `sentence-transformers` directly.
- For contracts, prefer RecursiveCharacterTextSplitter or the SemanticChunker above with larger `max_chars` and moderate `sim_threshold` (0.6-0.75) to avoid breaking clauses.
- Validate chunk outputs manually for the first few documents before bulk ingestion.
- Use TokenTextSplitter or a final token-count pass when you must guarantee token budget limits for your LLM.
